# Project 4:  Wrangle and Analyze Data 
### By Mirna Slim
#### The project's objective is to practice the 3 phases of data wrangling: gather, assess, and clean. 

## Data Set: WeRateDogs

#### The dataset used, wrangled, analyzed, and vicsualized is the tweet archive of Twitter user @dog_rates - also known as WeRateDogs - that rates people's dogs with a humorous comment about the dog. 
#### (Data files provided by Udacity will be used as a request to have a Twitter developer accpunt was denied).

## Sections of the Report:
 
•	Modules Imported
•	Data Source  
•	Gathering Data
•	Assessing data  
•	Cleaning data  
•	Storing, analyzing, and visualizing wrangled data  
•	Reporting on 1) your data wrangling efforts and 2) your data analyses and visualizations  



### Imported Modules

The following modules were imported to be used.  

In [2]:
#!pip install wptools;
#!pip install tweepy;

In [253]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wptools
import os
import requests
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
from PIL import Image
from io import BytesIO

%matplotlib inline

## Gathering Data

Request to get a Twitter Developer account was denied (despite using the language that Udacity provides), the Udacity Python code ('twitter_api.py') was used to access/gather data. The data files ('tweet_json.txt','twitter-archive-enhanced.csv', and 'image-predictions.tsv') built/provided by Udacity are used.  
There are 3 main data files that need to be gathered: 

### 1- The Enhanced Twitter Archive  
This file is provided by Udacity. It contains the WeRateDogs Twitter archive which consists of basic tweet data (not everything!) for 5000+ tweets. Udacity used the text column to extract:
- Tweet ID  
- dog name  
- dog rating    
- and dog "stage" (i.e. doggo, floofer, pupper, and puppo). 

The Enhanved Twitter Archive is prepared, by Udacity, from the JSON .txt file containing Twitter archive of the 5000+ tweets leaving only entries with ratings. A total of 2356 records/entries exist in the Enhanved Twitter Archive file. 

### 2- JSON.txt file
Ideally, the following steps would have been done to construct the JSON:  
1- Use tweet IDs in the WeRateDogs Twitter archive to query the Twitter API for each tweet's JSON data using Python's Tweepy library,   
2- Store each tweet's entire set of JSON data in a file called tweet_json.txt file (each tweet's JSON data should be written to its own line in a loop).  
3- Read the JSON file into a pandas DataFrame containing, at minimum, tweet ID, retweet count, and favorite count. 

### 3- The 'Iimage-prediction.tsv" file
The file contains image URLs and dog breed predictions of the various dogs in the archinve. A Neural Network was used. This file (image_predictions.tsv) is hosted on Udacity's servers and was downloaded programmatically using the Requests library 
Enhanced Twitter Archive

- p1 is the algorithm's #1 prediction for the image in the tweet.
- p2 and p3 are the algorithm's second and third most likely predictions.  
- p#_conf is how confident the algorithm is in each corresponding prediction. 
- p#_dog is whether or not the # prediction is a breed of dog → TRUE or FALSE


#### The following 3 cells are provided by Udacity as a Python code to access data if one has no Twitter API account. They are added in this notebook for completness. 

In [ ]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)


### Summary Data Gathering  

#### 1- The Enhanced Twitter Archive is used as provided.  
#### 2- The JSON .txt file is used as provided by Udacity (no need to build it from scratch).  
(refrence to learn how to extract tweets using tweepy: https://fairyonice.github.io/extract-someones-tweet-using-tweepy.html)
#### 3- The Image prediction file was imported and saved as a tsv file as follows:

In [32]:
# downloading the tweet image-prediction file programmatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

response

<Response [200]>

In [33]:
# save to file
path = 'C:/Users/Mirna.Slim/UdacityCurrent/Project4'
with open(os.path.join(path,
                       url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

===============================================================================================================================

### Open Data Files and Save to Dataframes  

In [397]:
# open the "Enhanced_twitter_Archive"
archive = pd.read_csv('twitter-archive-enhanced.csv')

In [398]:
# read the tweet_json.txt file:
with open('tweet_json.txt', 'r') as f:
    tweet_json = pd.read_json(f, lines=True)
tweet_json = tweet_json[['id','favorite_count','retweet_count']]
tweet_json.head(1)

,id,favorite_count,retweet_count
0,892420643555336193,39467,8853


In [399]:
image_prediction = pd.read_csv('image-predictions.tsv', sep='\t' )

In [400]:
archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,8.924210e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


-------------------------------------------------------------------------------------------------------------------------------

## Assessing Data
After gathering each of the above data files and open them as dataframes, the data is assessed visually and programmatically for quality and tidiness issues.  

### Visually  
Each data file can be opened and the various series, series' headers, data/entries can be inspected. A summary of the issues identified is provided below. 

### Programmatically
Inspectiion and data assessment is also done programmatically using functions like:
- .info(), .describe(): to give summaries of the data/series in the datasets.  
- .isnull().sum(): to check for missing data.  
- .duplicate(), .series.value_counts(): to see if entries belonging to the the same tweet ID are duplicated.  

### Programmatic Assessment of Enhanced Archive Dataframe

In [71]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   float64
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [72]:
archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455128e+17,2.015385e+16,7.720221e+17,1.241437e+16,13.126486,10.455433
std,6.856706e+16,7.583419e+16,1.253546e+17,6.236131e+16,9.597227e+16,45.876648,6.745237
min,6.660210e+17,6.660000e+17,1.185634e+07,6.660000e+17,7.832140e+05,0.000000,0.000000
25%,6.783992e+17,6.760000e+17,3.086374e+08,7.190000e+17,4.196984e+09,10.000000,10.000000
50%,7.196275e+17,7.035000e+17,4.196984e+09,7.800000e+17,4.196984e+09,11.000000,10.000000
75%,7.993375e+17,8.260000e+17,4.196984e+09,8.200000e+17,4.196984e+09,12.000000,10.000000
max,8.924210e+17,8.860000e+17,8.410000e+17,8.870000e+17,7.870000e+17,1776.000000,170.000000


In [73]:
# check for entry duplications
archive.tweet_id.duplicated().sum()

7

In [74]:
archive.tweet_id.value_counts().head(10)

6.757070e+17    2
8.322160e+17    2
6.737160e+17    2
6.747430e+17    2
6.751470e+17    2
6.675510e+17    2
8.190150e+17    2
7.276860e+17    1
8.347860e+17    1
8.703090e+17    1
Name: tweet_id, dtype: int64

In [75]:
# check for missing data/entries
archive.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [76]:
archive.rating_numerator.sort_values()

315        0
1016       0
2335       1
2261       1
2338       1
        ... 
2074     420
188      420
189      666
313      960
979     1776
Name: rating_numerator, Length: 2356, dtype: int64

-------------------------------------------------------------------------------------------------------------------------------

### Programmatic Assessment of tweet_json Dataframe

In [77]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2354 non-null   int64
 1   favorite_count  2354 non-null   int64
 2   retweet_count   2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [78]:
tweet_json.describe()

,id,favorite_count,retweet_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,8080.968564,3164.797366
std,6.852812e+16,11814.771334,5284.770364
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,1415.000000,624.500000
50%,7.194596e+17,3603.500000,1473.500000
75%,7.993058e+17,10122.250000,3652.000000
max,8.924206e+17,132810.000000,79515.000000


In [79]:
# check for entry duplications
tweet_json.id.duplicated().sum()

0

In [80]:
tweet_json.id.value_counts().head(10)

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
700462010979500032    1
780858289093574656    1
699775878809702401    1
880095782870896641    1
760521673607086080    1
Name: id, dtype: int64

In [81]:
# check for missing data/entries
tweet_json.isnull().sum()

id                0
favorite_count    0
retweet_count     0
dtype: int64

In [82]:
tweet_json.retweet_count.sort_values()

290         0
1293        2
273         3
341         3
112         3
        ...  
816     52360
1077    52360
259     56625
533     56625
1037    79515
Name: retweet_count, Length: 2354, dtype: int64

In [83]:
tweet_json.favorite_count.sort_values()

484          0
585          0
164          0
588          0
909          0
         ...  
134     106827
533     107015
65      107956
1037    131075
412     132810
Name: favorite_count, Length: 2354, dtype: int64

-------------------------------------------------------------------------------------------------------------------------------

### Programmatic Assessment of Image Prediction Dataframe

In [84]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [85]:
image_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [86]:
# check for entry duplications
image_prediction.tweet_id.duplicated().sum()

0

In [87]:
# check for entry duplications
image_prediction.jpg_url.duplicated().sum()

66

In [88]:
image_prediction.tweet_id.value_counts().head(10)

685532292383666176    1
826598365270007810    1
692158366030913536    1
714606013974974464    1
715696743237730304    1
776477788987613185    1
772114945936949249    1
699775878809702401    1
780858289093574656    1
700462010979500032    1
Name: tweet_id, dtype: int64

In [89]:
# check for missing data/entries
image_prediction.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [90]:
image_prediction.p1.sort_values()

1561         Afghan_hound
1855         Afghan_hound
1458         Afghan_hound
1804         Afghan_hound
446     African_crocodile
              ...        
253           wood_rabbit
1831         wooden_spoon
932                  wool
246                  wool
297                 zebra
Name: p1, Length: 2075, dtype: object

-------------------------------------------------------------------------------------------------------------------------------

## Quality Issues

### Enhanced Archive Data Set  
2,356 data entries in the dataset. The source column does not seem to add any value to differentiate tweets. It can be deleted.
The "text" column has already been analyzed by Udacity; all needed information extracted into separate columns. It can be deleted or not merged into the final cleaned dateframe. Statistical analyses can be done on Dog "stage" and rating ratio (numerator/denominator).  

1- Remove tweet entries prio to Agust 1, 2017.  

2- Unoriginal tweets  
- 78 entries of replies to status_id and user to remove
- 181 entries of re-tweets that should be removed  

3- Missing data  
- missing expanded urls  

4- 7 Duplicate entries and tweet_id. these may be indicating replies and retweets! Recheck for duplication after data cleaning.  
5- Tweet_id is wrong; it should be a string not a number written in a scientific format.  

6- Dtypes of rating numerators and denominators should be a float.  

7- Dog Names:wrong or missing.
   a- entries with one letter: e.g., "a"
   b- no names ("None")
   c- weird characters


### tweet_json.txt  
1- 2354 total entries --> 2 missing tweets!?  
2- Column header of tweet_id is "id" unlike header's name in the archive data.  
3- Dtype of "id" series is wrong; it is float. It should be a string.  


### image_prediction.tsv  
A total of 2.075 entries. 

1- Missing data.  
2- Tweet_id format should not be numerical. It should be a string.  
3- Dog breeds starting with lower and upper cases. Change all to lower case to be consistence.  
4- Not all images are for dogs. Remove entries that do not correspond to dogs.  
5- Extract/work only the predictions with the highest confidence.   
6- Duplicated image url for different tweet_ids.  




## Tidiness Issues  

### All Data sets  
1- Merging all datasets in one clean dataframe/one table with only important columns.  
2- Tweet_id as separate entry in all dataframes.  
3- Create a list of all columns and then find out duplicated columns

### Enhanced Archive Data Set  
1- Joining dog "stages" into one column instead of having 4 columns. 

--------------------------------------------------------------------------------------------------------------------------

## Data Cleaning  

In [91]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   float64
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

### Copy Dataframes

In [462]:
archive_clean = archive.copy()
tweet_json_clean = tweet_json.copy()
image_prediction_clean = image_prediction.copy()

### Cleaning Quality Issues:

### 1- Remove Unoriginal Tweet Entries  

#### Define  
Remove all entries pertaining to replies to and retweeted tweet.

#### Code

In [463]:
archive_clean = archive_clean[archive_clean['in_reply_to_status_id'].isnull()]
archive_clean = archive_clean[archive_clean['retweeted_status_id'].isnull()]

#### Test

In [464]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2097 non-null   float64
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   int64  
 11  rating_denominator          2097 non-null   int64  
 12  name                        2097 non-null   object 
 13  doggo                       2097 

### 2- Remove Duplicates   

#### Define  
After removing unoriginal tweets, check for tweet_id duplicates and remove them. 

In [465]:
# check for entry duplications
archive_clean.tweet_id.duplicated().sum()

1

In [466]:
archive_clean.tweet_id.value_counts()

6.751470e+17    2
7.184600e+17    1
6.681420e+17    1
6.722640e+17    1
6.709960e+17    1
               ..
8.892790e+17    1
6.729950e+17    1
8.918150e+17    1
7.783830e+17    1
7.787650e+17    1
Name: tweet_id, Length: 2096, dtype: int64

#### Code

In [467]:
archive_clean.tweet_id.drop_duplicates(inplace=True)

#### Test

In [468]:
# check for entry duplications
archive_clean.tweet_id.duplicated().sum()

0

### 3- Remove Rows with Missing Data (expanded URLs, )   

#### Define  
Remove rows that have missing "expanded_urls'.  
archive_clean.info() indicates that there are still 3 entries with no expanded url specified. 

#### Code

In [469]:
archive_clean.expanded_urls.dropna( inplace=True)

#### Test

In [470]:
# check for entry duplications
archive_clean.expanded_urls.isnull().sum()

0

Check if the image-prediction dataframe has series with missing data. 

In [471]:
# check for entry duplications
image_prediction_clean.jpg_url.isnull().sum()

0

### 4- Change Tweet ID Formats  

#### Define  
Change the tweet_id format to string instead of it being a flloat written in scientific format in all dataframes. 

#### Code

In [472]:
archive_clean.tweet_id = archive_clean.tweet_id.astype(str)

In [473]:
tweet_json_clean.id = tweet_json_clean.id.astype(str)

In [474]:
image_prediction_clean.tweet_id = image_prediction_clean.tweet_id.astype(str)

#### Test

In [475]:
archive_clean.tweet_id.dtype;
tweet_json_clean.id.dtype;
image_prediction_clean.tweet_id.dtype;

In [476]:
#archive_clean.info();
#tweet_json.info();
#image_prediction.info();

### 5- Change Rating (Numerator, Denominator) Formats  

#### Define  
Change the format of the rating entries to float.  

#### Code

In [477]:
archive_clean.rating_numerator = archive_clean.rating_numerator.astype(float)
archive_clean.rating_denominator = archive_clean.rating_denominator.astype(float)

#### Test

In [478]:
archive_clean.rating_numerator.dtype;
archive_clean.rating_denominator.dtype

dtype('float64')

### 6- Rename Series Header  

#### Define  
Rename the tweet id column in the tweet_json dataframe. Change header form 'id' to 'tweet_id'.   

#### Code

In [479]:
tweet_json_clean.rename(index=str, columns={'id':'tweet_id'}, inplace=True);

#### Test

In [480]:
tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   favorite_count  2354 non-null   int64 
 2   retweet_count   2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 73.6+ KB


### 7- Dog Breed Starting Lower and Upper Case  

#### Define  
The predicted dog breeds in the image_prediction file start with either lower or upper case.   

#### Code

In [481]:
image_prediction_clean.p1 = image_prediction_clean.p1.str.lower()
image_prediction_clean.p2 = image_prediction_clean.p2.str.lower()
image_prediction_clean.p3 = image_prediction_clean.p3.str.lower()

#### Test

In [482]:
image_prediction_clean.head(10);

### 8- Remove Image Entries Not Showing Dogs   

#### Define  
The iamge_prediction file has entries where the images are not indicating a dog. Remove images not showing dogs in the algorithim's #1 prediction (p1 and p1_dog) from dataframe.  

#### Code

In [483]:
image_prediction_clean = image_prediction_clean.loc[image_prediction_clean.p1_dog, :]

In [484]:
#image_prediction_clean = image_prediction_clean[image_prediction_clean['p1_dog'] == 'FALSE']

In [485]:
image_prediction_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 0 to 2073
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1532 non-null   object 
 1   jpg_url   1532 non-null   object 
 2   img_num   1532 non-null   int64  
 3   p1        1532 non-null   object 
 4   p1_conf   1532 non-null   float64
 5   p1_dog    1532 non-null   bool   
 6   p2        1532 non-null   object 
 7   p2_conf   1532 non-null   float64
 8   p2_dog    1532 non-null   bool   
 9   p3        1532 non-null   object 
 10  p3_conf   1532 non-null   float64
 11  p3_dog    1532 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 124.2+ KB


In [486]:
image_prediction_clean.p1_dog.sum()

1532

In [487]:
# check for entry duplications
image_prediction_clean.jpg_url.duplicated().sum()

52

Even after removing imagees not showing dogs, there are still 52 duplicated image URLs in the dataframe. 

In [488]:
# check for entry duplications
image_prediction_clean.tweet_id.duplicated().sum()

0

In [489]:
## check for entry duplications
#image_prediction_clean.jpg_url.duplicated().index

### 9- Correct Dog Names   

#### Define  
There are some tweets where the dog name is either 'None' or weird character/adjective ('a' or 'an', 'quite', 'the','such'). Remove these entries from data set if needed. 

In [490]:
archive_clean.name.value_counts()

None          603
a              55
Charlie        11
Lucy           11
Oliver         10
             ... 
Nimbus          1
officially      1
Chevy           1
Kenny           1
Perry           1
Name: name, Length: 955, dtype: int64

In [491]:
wrong_name = ['None','a','an','the','such','quite']
for i in wrong_name:
    len(archive_clean[archive_clean['name'] == i])
    print (len(archive_clean[archive_clean['name'] == i]))

603
55
6
8
1
3


#### Code

In [492]:
archive_nobadnames = archive_clean.copy()

In [493]:
wrong_name = ['None','a','an','the','such','quite']
for i in wrong_name:
    archive_nobadnames.drop(archive_nobadnames[archive_nobadnames['name'] == i].index, inplace = True) 

#### Test

In [494]:
archive_nobadnames.name.value_counts()

Charlie       11
Lucy          11
Oliver        10
Cooper        10
Tucker         9
              ..
officially     1
Chevy          1
Kenny          1
Brandonald     1
Perry          1
Name: name, Length: 949, dtype: int64

In [495]:
#BAD.  this leaves only rows with dog name = 'a'.
#archive_clean = archive_clean[archive_clean['name'] == 'a']

### Cleaning Tidiness Issues:

### 1- Combine Dog Stage Columns   

#### Define  
In the twitter_archive_enhanced file, the dog stage data occur, needlessly, in 4 columns. We need to merge all 4 columns into 1.  

#### Code  

Hint obtained from a mentor's answer in Udacity help page. 

In [496]:
# replace all 'None' entries for dog stage with '' 
archive_clean.doggo.replace('None', '', inplace=True)
archive_clean.floofer.replace('None', '', inplace=True)
archive_clean.pupper.replace('None', '', inplace=True)
archive_clean.puppo.replace('None', '', inplace=True)

In [497]:
# combining all dog stage columns in a new column called 'stage'
archive_clean['stage'] = archive_clean.doggo + archive_clean.floofer + archive_clean.pupper + archive_clean.puppo

In [498]:
# show some statistics on the most abumdant doge stage identified in the data
archive_clean.stage.value_counts()

                1761
pupper           221
doggo             72
puppo             23
doggopupper        9
floofer            9
doggofloofer       1
doggopuppo         1
Name: stage, dtype: int64

In [499]:
# further cleaning in case thre are entries with double stages indicated (not the case in our cleaned archive dataframe)
archive_clean.loc[archive_clean.stage == 'doggopupper', 'stage'] = 'doggo,pupper'
archive_clean.loc[archive_clean.stage == 'doggopuppo', 'stage'] = 'doggo,puppo'
archive_clean.loc[archive_clean.stage == 'doggofloofer', 'stage'] = 'doggo,floofer'

In [500]:
archive_clean.loc[archive_clean.stage == '', 'stage'] = np.nan

#### Test

In [501]:
# drop unneeded columns from the dataframe
archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

In [502]:
archive_clean.stage.value_counts()

pupper           221
doggo             72
puppo             23
doggo,pupper       9
floofer            9
doggo,floofer      1
doggo,puppo        1
Name: stage, dtype: int64

In [505]:
archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,stage
0,8.92421e+17,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,Phineas,NaN
1,8.92177e+17,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13.0,10.0,Tilly,NaN


In [506]:
archive_clean.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2096 non-null   object 
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   float64
 11  rating_denominator          2097 non-null   float64
 12  name                        2097 non-null   object 
 13  stage                       336 n

### 2- Merge Dataframes  

Before moving to analyses and visualization:  
1- Uneeded columns will be dropped from dataframe (twitter_enhanced_archinve and image-prediction). Only 3 columns ('tweet_id', 'retweet_count', and 'favorite_count'will be useful from the tweet_json file.  

2- All dataframes will be merged into one final dataframe.  

#### 2a - Define  
Drop cloumns from archive_clean

#### 2a- Code

In [507]:
archive_clean2 = archive_clean.copy()

In [508]:
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2096 non-null   object 
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   float64
 11  rating_denominator          2097 non-null   float64
 12  name                        2097 non-null   object 
 13  stage                       336 n

In [509]:
# drop unneeded columns from the archive dataframe 
archive_clean2.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'text'], axis=1, inplace=True)

In [510]:
# drop unneeded columns from the archive dataframe 
archive_clean2.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace=True)

#### 2a- Test

In [511]:
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2096 non-null   object 
 1   timestamp           2097 non-null   object 
 2   expanded_urls       2094 non-null   object 
 3   rating_numerator    2097 non-null   float64
 4   rating_denominator  2097 non-null   float64
 5   name                2097 non-null   object 
 6   stage               336 non-null    object 
dtypes: float64(2), object(5)
memory usage: 131.1+ KB


#### 2b - Define  
Drop uneeded cloumns from image-prediction. Keep only predictions with high confidence (p1, p1_dog). 

In [512]:
image_prediction_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 0 to 2073
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1532 non-null   object 
 1   jpg_url   1532 non-null   object 
 2   img_num   1532 non-null   int64  
 3   p1        1532 non-null   object 
 4   p1_conf   1532 non-null   float64
 5   p1_dog    1532 non-null   bool   
 6   p2        1532 non-null   object 
 7   p2_conf   1532 non-null   float64
 8   p2_dog    1532 non-null   bool   
 9   p3        1532 non-null   object 
 10  p3_conf   1532 non-null   float64
 11  p3_dog    1532 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 124.2+ KB


#### 2b- Code

In [513]:
image_prediction_clean2 = image_prediction_clean.copy()

In [514]:
# drop unneeded columns from the archive dataframe 
image_prediction_clean2.drop(['p2', 'p3', 'p2_conf', 'p3_conf', 'p1_dog', 'p2_dog', 'p3_dog'], axis=1, inplace=True)

#### 2b- Test

In [515]:
image_prediction_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 0 to 2073
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1532 non-null   object 
 1   jpg_url   1532 non-null   object 
 2   img_num   1532 non-null   int64  
 3   p1        1532 non-null   object 
 4   p1_conf   1532 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 71.8+ KB


#### Check the number of tweet_id in each df to see how to merge the dfs.

In [516]:
tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   favorite_count  2354 non-null   int64 
 2   retweet_count   2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 73.6+ KB


In [517]:
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2096 non-null   object 
 1   timestamp           2097 non-null   object 
 2   expanded_urls       2094 non-null   object 
 3   rating_numerator    2097 non-null   float64
 4   rating_denominator  2097 non-null   float64
 5   name                2097 non-null   object 
 6   stage               336 non-null    object 
dtypes: float64(2), object(5)
memory usage: 131.1+ KB


#### 2c - Define  
Merge dataframes into one.

#### 2c- Code

#### Concat is not the right thing to do as dfs are added on top of each others. merge is a better choice:
referecen 'https://realpython.com/pandas-merge-join-and-concat/#how-to-merge'

In [518]:
#tweets_final1 = pd.concat([archive_clean2,image_prediction_clean2,tweet_json_clean])

In [519]:
image_prediction_clean2['tweet_id'] = pd.to_numeric(image_prediction_clean2['tweet_id'], errors='coerce')
image_prediction_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 0 to 2073
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1532 non-null   int64  
 1   jpg_url   1532 non-null   object 
 2   img_num   1532 non-null   int64  
 3   p1        1532 non-null   object 
 4   p1_conf   1532 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 71.8+ KB


In [520]:
tweet_json_clean['tweet_id'] = pd.to_numeric(tweet_json_clean['tweet_id'], errors='coerce')
tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   favorite_count  2354 non-null   int64
 2   retweet_count   2354 non-null   int64
dtypes: int64(3)
memory usage: 73.6+ KB


In [521]:
archive_clean2['tweet_id'] = pd.to_numeric(archive_clean2['tweet_id'], errors='coerce')
#archive_clean2.tweet_id = archive_clean2.tweet_id.astype(int)
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2096 non-null   float64
 1   timestamp           2097 non-null   object 
 2   expanded_urls       2094 non-null   object 
 3   rating_numerator    2097 non-null   float64
 4   rating_denominator  2097 non-null   float64
 5   name                2097 non-null   object 
 6   stage               336 non-null    object 
dtypes: float64(3), object(4)
memory usage: 131.1+ KB


In [523]:
archive_clean2.tweet_id = archive_clean.tweet_id.astype(str)
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2097 non-null   object 
 1   timestamp           2097 non-null   object 
 2   expanded_urls       2094 non-null   object 
 3   rating_numerator    2097 non-null   float64
 4   rating_denominator  2097 non-null   float64
 5   name                2097 non-null   object 
 6   stage               336 non-null    object 
dtypes: float64(2), object(5)
memory usage: 131.1+ KB


In [524]:
archive_clean2['tweet_id'] = pd.to_numeric(archive_clean2['tweet_id'], errors='coerce')
archive_clean2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2096 non-null   float64
 1   timestamp           2097 non-null   object 
 2   expanded_urls       2094 non-null   object 
 3   rating_numerator    2097 non-null   float64
 4   rating_denominator  2097 non-null   float64
 5   name                2097 non-null   object 
 6   stage               336 non-null    object 
dtypes: float64(3), object(4)
memory usage: 131.1+ KB


In [ ]:
archive_clean2.tweet_id = archive_clean2.tweet_id.astype(str)

archive_clean2.info()

In [522]:
archive_clean2.to_csv('archive_clean2.csv')

In [385]:
tweets_final2 = pd.merge(archive_clean2, image_prediction_clean2, how='inner', on="tweet_id")

In [386]:
tweets_final2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            0 non-null      float64
 1   timestamp           0 non-null      object 
 2   expanded_urls       0 non-null      object 
 3   rating_numerator    0 non-null      float64
 4   rating_denominator  0 non-null      float64
 5   name                0 non-null      object 
 6   stage               0 non-null      object 
 7   jpg_url             0 non-null      object 
 8   img_num             0 non-null      int64  
 9   p1                  0 non-null      object 
 10  p1_conf             0 non-null      float64
dtypes: float64(4), int64(1), object(6)
memory usage: 0.0+ bytes


In [ ]:
#tweets_final2 = pd.concat([archive_clean2,image_prediction_clean2,tweet_json_clean])

## Analyses and Visualization  

Storing, Analyzing, and Visualizing Data for this Project
Store the clean DataFrame(s) in a CSV file with the main one named twitter_archive_master.csv. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

Analyze and visualize your wrangled data in your wrangle_act.ipynb Jupyter Notebook. At least three (3) insights and one (1) visualization must be produced.

### Store Cleaned Dataframe

In [345]:
tweets_final.to_csv('dog_tweet_df_final.csv')

### Preliminary Insights!

- Tweets favorited by most users?

- Common dog names  

- easily identified dog breeds: What are the most common breeds found by the neural network?  

- correlating breed to ratings

In [ ]:
##If you want to see only the rows that contains the NaN values you could do:
#data_frame[data_frame.iloc[:, insert column number here]=='NaN']

In [ ]:
# search for specific entry
archive[archive.series1 == 'entry'].series2

In [50]:
#list(archive)
#archive.sample(5)
#patients[patients.address.duplicated()]

In [ ]:
# rename column 'id' to 'tweet_id'
tweet_likes.rename(columns={'id':'tweet_id'}, inplace=True)

### Reporting for this Project
Create a 300-600 word written report called wrangle_report.pdf or wrangle_report.html that briefly describes your wrangling efforts. This is to be framed as an internal document.

Create a 250-word-minimum written report called act_report.pdf or act_report.html that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

Both of these documents can be created in separate Jupyter Notebooks using the Markdown functionality of Jupyter Notebooks, then downloading those notebooks as PDF files or HTML files (see image below). You might prefer to use a word processor like Google Docs or Microsoft Word, however.



•	twitter_archive_master.csv: combined and cleaned data